### Gradio for NLP tasks

In this notebook, we can see how to use the combination of two powerful libraries, [HuggingFace](https://huggingface.co/) and [Gradio](https://www.gradio.app/), to build Generative AI applications. Gradio allows to quickly create a simple web interface to make the access to most LLMs models more user friendly.

In [ ]:
# libraries
from transformers import pipeline       # huggingface
import gradio as gr                     # gradio

#### 1. Building a text summarization app

With `HuggingFace` is simple to download a pretrained model for a specific task (here, summarization) and use it for inference.

In [ ]:
# initialize pipeline
get_completion = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
def summarize(my_input):
    my_output = get_completion(my_input)
    return my_output[0]['summary_text']

In [ ]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

With Gradio's `gr.Interface` is possible to write a simple UI where the user can write an input and get an output, ideally without the need to see the code behind.

In [ ]:
gr.close_all()                                                    # to close existing instances
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")  # basic interface
demo.launch(share=False)                                          # launch locally in the notebook
#demo.launch(share=True, server_port=int(os.environ['PORT1']))    # to launch online

Let's improve the interface.

In [ ]:
gr.close_all()
demo = gr.Interface(
    fn=summarize, 
    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
    outputs=[gr.Textbox(label="Result", lines=3)],
    title="Text summarization using LLMs",
    description="Summarize any text using the `facebook/bart-large-cnn` model from `HuggingFace`."
)
demo.launch(share=False)

#### 2. Building a named entity recognition app

**Named-entity recognition (NER)** (also known as *(named) entity identification*, *entity chunking*, and *entity extraction*) is a task that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories (e.g., person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.).

This task has become way easier since LLMs are around so, we'll show here how to use HuggingFace + Gradio to generate a handy UI for a NER app. 

**Note** that the final aim is have these apps running and available online. This can be achieved using the Huggingface [Inference Endpoint](https://huggingface.co/inference-endpoints), but here we'll do everything locally.

The model chosen is `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

In [ ]:
get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(my_input):
    my_output = get_completion(my_input)
    return {"text": my_input, "entities": my_output}

If we have an endpoint on HuggingFace, then:
```py
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)
```

In [ ]:
gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Text to find entities", lines=2)], # ner() can take more than one inputs...
    outputs=[gr.HighlightedText(label="Text with entities")],    # ...and return more than one outputs
    title="Named Entity Recognition",
    description="NER is performed using the `dslim/bert-base-NER` model from `HuggingFace`",
    examples=[                                                   # provide example prompts to show how the model works 
        "Deutsche Bahn is the national railway company of Germany", 
        "The Sun is the closest star to Earth"
    ],
    allow_flagging="never",                                      # see Gradio website to understand what this does
)
demo.launch(share=False)#, server_port=int(os.environ['PORT3']))

`Gradio` allows for lot of flexibility when calling the `fn` function. Here we'll add a helper function that makes the output a bit more consistent.

In [ ]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            last_token = merged_tokens[-1]         # if current token continues the entity of the last one, merge them
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            merged_tokens.append(token)            # otherwise, add the token to the list
    return merged_tokens

def modified_ner(my_input):
    my_output = get_completion(my_input)
    merged_tokens = merge_tokens(my_output)
    return {"text": my_input, "entities": merged_tokens}

demo = gr.Interface(
    fn=modified_ner,
    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
    outputs=[gr.HighlightedText(label="Text with entities")],
    title="Named Entity Recognition",
    description="NER is performed using the `dslim/bert-base-NER` model from `HuggingFace`",
    allow_flagging="never",
    examples=[                                                  
        "Deutsche Bahn is the national railway company of Germany, one member state of the European Union", 
        "The Sun is the closest star to Earth and it moves around the Galactic Center of the Milky Way"
    ],
)
demo.launch(share=False)

In [ ]:
# remember to close all the ports
gr.close_all()

### Acknowledgements

Thanks to DeepLearning.AI and Gradio for the courses that inspired this notebook.